# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are:

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

# Getting the words

In [99]:
freq = dict()
words = set()
with open("big.txt", 'r') as f:
    text = f.read().lower()
    i = 0
    word = []
    while i < len(text):
        if text[i].isalpha():
            word.append(text[i])
            i += 1
        else:
            if len(word) != 0:
                words.add(''.join(word))
                freq[''.join(word)] = freq.get(''.join(word), 0) + 1
            i += 1
            word = []

In [100]:
len(words)

29157

In [101]:
with open("fivegrams (2).txt", 'r') as f:
    for line in f:
        n, w1, w2, w3, w4, w5 = line.split()
        words.add(w1)
        freq[w1] = freq.get(w1, 0) + 1
        words.add(w2)
        freq[w2] = freq.get(w2, 0) + 1
        words.add(w3)
        freq[w3] = freq.get(w3, 0) + 1
        words.add(w4)
        freq[w4] = freq.get(w4, 0) + 1
        words.add(w5)
        freq[w5] = freq.get(w5, 0) + 1

In [102]:
len(words)

40187

# Getting all the typos

In [103]:
import string


def typos(words):
    res = {}
    for word in words:
        # delete
        for i in range(len(word)):
            new_word = word[:i] + word[i+1:]
            if new_word not in res:
                res[new_word] = set()
            res[new_word].add(word)
        # transpose
        for i in range(len(word)-1):
            new_word = word[:i] + word[i+1] + word[i] + word[i+2:]
            if new_word not in res:
                res[new_word] = set()
            res[new_word].add(word)
        # replace
        for i in range(len(word)):
            for character in string.ascii_lowercase:
                new_word = word[:i] + character + word[i+1:]
                if new_word not in res:
                    res[new_word] = set()
                res[new_word].add(word)
        # insert
        for i in range(len(word) + 1):
            for character in string.ascii_lowercase:
                new_word = word[:i] + character + word[i:]
                if new_word not in res:
                    res[new_word] = set()
                res[new_word].add(word)
    return res

# get first itaration typos

In [104]:
err2word1 = typos(words)

# BiGram

In [105]:
bigrams = {}
with open("bigrams (2).txt", encoding='latin-1') as f:
    cn = 0
    for line in f:
        n, w1, w2 = line.split()
        if w1 not in bigrams:
            bigrams[w1] = {}
        bigrams[w1][w2] = int(n)
        freq[w1] = freq.get(w1, 0) + int(n)
        freq[w2] = freq.get(w2, 0) + int(n)

# Main algorithm

In [106]:
def get_candidates(word):
    if word in err2word1:
        return err2word1[word]
    err2 = typos(typos([word]).keys()).keys()
    res = []
    for error_word in err2:
        if error_word in words:
            res.append(error_word)
    return set(res)


def my_main(sentance: list[str]) -> list[str]:
    for i in range(len(sentance)):
        if sentance[i] not in words:
            cands = list(get_candidates(sentance[i]))
            if len(cands) == 0:
                continue
            bgr = [0] * len(cands)
            bgl = [0] * len(cands)
            for j in range(1, len(cands)):
                if cands[j - 1] in bigrams and cands[j] in bigrams[cands[j - 1]]:
                    bgr[j] = bigrams[cands[j - 1]][cands[j]]
                    bgl[j-1] = bigrams[cands[j - 1]][cands[j]]
            bg_final = [bgr[j] + bgl[j] + freq.get(cands[j], 0) / 100_000_000 for j in range(len(cands))]
            sentance[i] = cands[bg_final.index(max(bg_final))]
    return sentance

In [107]:
my_main("dking sport".split())

['doing', 'sport']

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

#### 1

I got all word which I can get and their frequency

#### 2

I create func `tions` and first iteration errors for each word

#### 3

I create bigram by file `bigrams (2).txt`

#### 4

I write `my_main` function that:

* choosing candidates: consider candidates by priority - 0 errors, 1 errors, 2 errors (if errors more then 2, I don't change the word)

* for each candidate (if more than 1) I consider next and previous word in birgham, and if I have the pair in birgham I add frequency in sum

* if I have several candidates with equal sums, I compare it by their frequency of words (frequency / 100_000_000 -> (0 - 1))

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

# Create Dataset

In [108]:
true_text = list()
with open("True_spases.txt", 'r') as f:
    text = f.read().lower()
    i = 0
    word = []
    while i < len(text):
        if text[i].isalpha():
            word.append(text[i])
            i += 1
        else:
            if len(word) != 0:
                true_text.append(''.join(word))
            i += 1
            word = []
    if len(word) != 0:
        true_text.append(''.join(word))

In [109]:
error_text = list()
with open("error.txt", 'r') as f:
    text = f.read().lower()
    i = 0
    word = []
    while i < len(text):
        if text[i].isalpha():
            word.append(text[i])
            i += 1
        else:
            if len(word) != 0:
                error_text.append(''.join(word))
            i += 1
            word = []
    if len(word) != 0:
        error_text.append(''.join(word))

In [110]:
assert len(error_text) == len(true_text)

# Norvig's solution

In [111]:
import re
from collections import Counter

def get_words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(get_words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def norvigs_main(sentance: list[str]) -> list[str]:
    return [correction(word) for word in sentance]

In [112]:
norvigs_main("dking sport".split())

['king', 'sport']

# Compare

In [113]:
total_errors = sum([1 if error_text[i] != true_text[i] else 0 for i in range(len(true_text))])

In [114]:
total_errors

20323

In [90]:
norvigs_res = norvigs_main(error_text)

In [91]:
norvigs_errors = sum([1 if norvigs_res[i] != true_text[i] else 0 for i in range(len(true_text))])

In [116]:
my_res = my_main(error_text)

In [117]:
my_errors = sum([1 if my_res[i] != true_text[i] else 0 for i in range(len(true_text))])

In [118]:
print(f"norvig's accuracy: {norvigs_errors/total_errors*100}%")
print(f"my accuracy: {my_errors/total_errors*100}%")

norvig's accuracy: 36.864636126556114%
my accuracy: 37.36160999852384%


I expected a bigger difference, but it looks like I have a bad implementation or the norvig's approach is difficult to modify much :)

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)